In [14]:
import pandas as pd
import numpy as np
def data_cleaning(filename):
    """
    남산 도서관 장서 CSV데이터 전처리 함수
    :param filename: CSV 파일 이름
    """
    #파일을 데이터프레임으로 읽기
    ns_df = pd.read_csv(filename, low_memory=False)
    #NaN인 열 삭제
    ns_book = ns_df.dropna(axis=1, how='all')
    #대출건수를 합치기 위해 필요한 행만 추출하여 count_df 데이터프레임 생성
    count_df = ns_book[['도서명', '저자', 'ISBN', '권', '대출건수']]
    #도서명, 저자, ISBN, 권을 기준으로 대출건수를 groupby
    loan_count = count_df.groupby(by=['도서명', '저자', 'ISBN', '권'], dropna=False).sum()
    #원본 데이터프레임에서 중복된 행을 제외하고 고유한 행만 추출하여 복사
    dup_rows = ns_book.duplicated(subset=['도서명', '저자', 'ISBN', '권'])
    unique_rows = ~dup_rows
    ns_book3 = ns_book[unique_rows].copy()
    #도서명, 저자, ISBN, 권을 인덱스로 설정
    ns_book3.set_index(['도서명', '저자', 'ISBN', '권'], inplace=True)
    #loan_count에 저장된 누적 대출건수를 업데이트
    ns_book3.update(loan_count)
    #인덱스 재설정
    ns_book4 = ns_book3.reset_index()
    #원본 데이터프레임의 열 순서로 변경
    ns_book4 = ns_book4[ns_book.columns]
    return ns_book4

In [7]:
new_ns_book4 = data_cleaning('ns_202309.csv')

In [8]:
ns_book4 = pd.read_csv('ns_book4.csv', low_memory=False)
ns_book4.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0.0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0.0,2021-03-19
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0.0,2021-03-19
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0.0,2021-03-19
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0.0,2021-03-19


In [9]:
#누락된 값 변경
#'세트ISBN'열 대부분 비어있음. 누락된 값을 NaN가 아닌 빈 문자열('')로 변경
#loc() - 누락된 값을 원하는 값으로 변경 가능

set_isbn_na_rows = ns_book4['세트 ISBN'].isna() #누락된 값을 찾아 boolean배열로 반환
ns_book4.loc[set_isbn_na_rows, '세트 ISBN'] = ''#누락된 값을 빈 문자열로 변경
ns_book4['세트 ISBN'].isna().sum()              #누락된 값 개수 세기

0

In [10]:
#fillna()를 통해 더욱 쉽게 누락된 값 변경 가능
ns_book4.fillna('없음').isna().sum()

번호         0
도서명        0
저자         0
출판사        0
발행년도       0
ISBN       0
세트 ISBN    0
부가기호       0
권          0
주제분류번호     0
도서권수       0
대출건수       0
등록일자       0
dtype: int64

In [11]:
#ns_book4에 컬럼값을 지정해줌으로써 특정 열만 선택해서 바꿀수도 있음
ns_book4['부가기호'].fillna('없음').isna().sum()
#열이름 없이 개수만있는 판다스 시리즈 객체로 반환

0

In [12]:
#특정열의 NaN를 바꾸면서 전체 데이터프레임을 반환하려면 딕셔너리를 전달하면 됨
ns_book4.fillna({'부가기호':'없음'}).isna().sum()

번호              0
도서명           403
저자            198
출판사          4641
발행년도           14
ISBN            0
세트 ISBN         0
부가기호            0
권          321213
주제분류번호      19864
도서권수            0
대출건수            0
등록일자            0
dtype: int64

In [15]:
#누락된 값 변경(2) / replace()
#1. 단일 값 변경
#replace(원래 값, 새로운 값)
ns_book4.replace(np.nan, '없음').isna().sum()

번호         0
도서명        0
저자         0
출판사        0
발행년도       0
ISBN       0
세트 ISBN    0
부가기호       0
권          0
주제분류번호     0
도서권수       0
대출건수       0
등록일자       0
dtype: int64

In [16]:
#2. 다중값 변경
#replace([원래 값, 원래 값2], [새로운 값, 새로운 값2])
ns_book4.replace([np.nan, '2021'],['없음','21']).head(2)

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,21,9788937444319,,없음,없음,없음,1,0.0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,21,9791190123969,,없음,없음,없음,1,0.0,2021-03-19


In [17]:
#딕셔너리 형식으로 전달 가능
#replace({원래 값:새로운 값, 원래 값2:새로운 값2})
ns_book4.replace({np.nan:'없음', '2021':'21'}).head(2)

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,21,9788937444319,,없음,없음,없음,1,0.0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,21,9791190123969,,없음,없음,없음,1,0.0,2021-03-19


In [18]:
#중첩된 딕셔너리도 가능
#replace({열 이름: {원래 값:새로운 값}})
ns_book4.replace({'부가기호':{np.nan:'없음'}}).head(2)

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,,없음,NaN,NaN,1,0.0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,,없음,NaN,NaN,1,0.0,2021-03-19


In [21]:
ns_book4.replace({'발행년도':{'2021':'21'}})[100:102] #발행년도가 2018년도라 변경 X 이런경우 정규표현식을 사용

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
100,101,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",좋은씨앗,2018,9788958743019,,NaN,NaN,234.9,1,1.0,2021-03-15
101,102,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",부키,21,9788960518483,,NaN,NaN,325.1,1,0.0,2021-03-15


In [24]:
#정규표현식
#정규표현식에서 숫자를 나타내는 기호 - \d
#4자리 연도에 해당하는 표현 = \d\d\d\d
#뒤 두자리만 하나의 그룹으로 묶을때는 \d\d(\d\d)로 표현
#그룹을 나타낼때는 \1\2로 사용
ns_book4.replace({'발행년도':{r'\d\d(\d\d)':r'\1'}}, regex=True)[100:102] #regex매개변수옵션을 True로 지정해줌으로써 정규식을 사용한다는 의미
#r문자는 정규표현식을 다른 문자열과 구분하기 위해 접두사처럼 사용

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
100,101,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",좋은씨앗,18,9788958743019,,NaN,NaN,234.9,1,1.0,2021-03-15
101,102,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",부키,21,9788960518483,,NaN,NaN,325.1,1,0.0,2021-03-15


In [27]:
#정규식이 반복될 때 중괄호를 사용하여 개수를 지정할 수 있음
ns_book4.replace({'발행년도':{r'\d{2}(\d{2})':r'\1'}}, regex=True)[100:102]

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
100,101,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",좋은씨앗,18,9788958743019,,NaN,NaN,234.9,1,1.0,2021-03-15
101,102,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",부키,21,9788960518483,,NaN,NaN,325.1,1,0.0,2021-03-15


In [28]:
#문자열정규식 너무 어려움 공부 다시 해야함.....제발 하자

In [ ]:
#잘못된 값 바꾸기
